# Embeddings con LSTM



In [105]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import Counter
import random
from itertools import chain
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
cd ..

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal


In [3]:
from helpers.boletines import get_sentences

In [4]:
cd Embeddings/

c:\Users\Usuario.000\Documents\Facultad\Git\2020-2\APIT-2020-2\ProyectoFinal\Embeddings


## Definición de constantes

In [5]:
# Número de GPUs disponibles. Usar 0 para modo CPU.
ngpu = 1

# Semilla a usar en los generadores de números aleatorios
SEED = 42069
# SEED = random.randint(1, 10000) # En caso de requerir más resultados
random.seed(SEED)
torch.manual_seed(SEED)

print("Random Seed: ", SEED)

Random Seed:  42069


Selecciono el tipo de dispositivo a utilizar (gpu o cpu)

In [6]:
# Decide si queremos correr en gpu o cpu
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
device

device(type='cuda', index=0)

## Obtener corpus

In [7]:
uri = open('./../mongo_uri.txt', 'r', encoding='utf-8').read()

In [8]:
sentences = get_sentences(uri)

RAC
1. De_la_mano_de jóvenes innovadores , comienza Ricardo_Anaya el camino hacia la Presidencia_de_la_República .
377. Finalmente , sostuvo que López_Obrador es “ el espanta inversiones ” , porque su actitud provoca que estas se vayan , lo cual es muy delicado , porque cuando no hay inversión no crece la economía y no se generan empleos .
753. “ Sí , necesitamos cambiar ciertas piezas de el modelo económico , estamos en un momento crítico , donde tenemos que elegir entre volver a el pasado a_través_de un cómic o construir un futuro más incluyente , igualitario , a_partir_de propuestas series , razonadas , en donde los cómo están claros y los para qué , son parte principal de lo que se pretende hacer ” , enfatizó a el contrastar con la propuesta que hace López_Obrador .
1129. Yo quiero paz para Jalisco , yo quiero paz para México , y esa paz la voy a construir con las mujeres , esa paz la vamos a construir todos juntos ” , enfatizó .

AMLO
1. Asegura AMLO que en campaña todo será amor 

Desenvuelvo los grupos de enunciados en un único arreglo con todos los enunciados de todos los candidatos.

In [9]:
corpus = list(chain(*sentences.values()))

Paso los tokens a minúsculas para reducir el tamaño del vocabulario

In [10]:
corpus = [[w.lower() for w in sent] for sent in corpus]
print(f'El corpus consta de {len(corpus)} enunciados.')

El corpus consta de 6889 enunciados.


Selecciono k oraciones de prueba para el modelo

In [11]:
k = 1000
sentences = random.sample(corpus, k=k)
print('\n'.join([' '.join(sent) for sent in sentences[::k//10]]))

promete amlo que no cancelará , ni modificará las concesiones en radio y televisión .
además , las estancias que atiendan a ciertos sectores económicos como las maquiladoras , tendrán horario nocturno .
queremos externarle a andrés_manuel que no pierda su buen sentido de el humor , porque , andrés_manuel , ¡ vas a volver a perder ! ” , subrayó meade ante habitantes de la región de los_altos_de_jalisco , quienes se unieron en una sola voz para apoyar a su candidato con la porra ¡ pepe presidente !
meade congrega a más de 60_mil simpatizantes durante los eventos de el fin de semana .
y creo que la conclusión es que no ha fallado la gente , han fallado los gobiernos , porque hemos tenido gobiernos profundamente corruptos y gobiernos profundamente ineficaces y eso es lo que yo he querido cambiar ” .
josé_antonio_meade impulsará el desarrollo integral de niñas , niños y adolescentes , dijo .
este programa , detalló , consiste en acciones concretas para la primera infancia , desde la gestaci

## Indexar Corpus

Cuento los tokens en todas las oraciones e imprimo los 40 más frecuentes.

In [12]:
freq = sum([Counter(sent) for sent in sentences], Counter())
print(freq.most_common()[:-40:-1])

[('toma', 1), ('votarán', 1), ('escribió', 1), ('conciencia', 1), ('plurinominal', 1), ('canadiense', 1), ('adquirido', 1), ('55_millones_de_dólares', 1), ('señalado', 1), ('napoleón_gomez_urrutia', 1), ('contento', 1), ('personalizada', 1), ('formalización', 1), ('misma', 1), ('mueve', 1), ('siglos', 1), ('pasan', 1), ('repite', 1), ('usual', 1), ('interesantes', 1), ('productos', 1), ('comuneros', 1), ('bloqueó', 1), ('vano', 1), ('papantla', 1), ('amecameca', 1), ('slim', 1), ('saliera', 1), ('carlos_slim', 1), ('carlos_salinas', 1), ('posiblemente', 1), ('geografía', 1), ('mapa', 1), ('visita', 1), ('diputado', 1), ('comité_directivo_estatal_de_el_pri', 1), ('presiente', 1), ('mexe', 1), ('reabrir', 1)]


In [13]:
num_tokens = len(freq.keys())
num_tokens

5269

Agrego un diccionario para pasar de palabra a índice numérico

In [14]:
w_to_index = {
    w : ix
    for ix, (w, freq) in enumerate(freq.most_common())
    if freq > 1 # No toma en cuenta los hapax
}

In [15]:
BOS = '<BOS>'
EOS = '<EOS>'
UNK = '<UNK>'

ixBOS = num_tokens + 1
ixEOS = num_tokens + 2
ixUNK = num_tokens + 3

In [16]:
w_to_index[BOS] = ixBOS
w_to_index[EOS] = ixEOS
w_to_index[UNK] = ixUNK

Creo el diccionario inverso, para convertir de índices a palabras

In [17]:
index_to_w = [ w for w, ix in w_to_index.items() ]

In [18]:
# tests = random.sample(w_to_index.keys(), k=10)
# for w in tests:
#     ix = w_to_index[w]
#     assert index_to_w[ix] == w

Indexo todo el corpus

In [19]:
def w_to_index_unk(w):
    """
    Le asigna el token UNK a palabras que no aparezcan en el corpus
    """
    try:
        return w_to_index[w] 
    except KeyError:
        return ixUNK

In [20]:
sentences_ix = [
    [ w_to_index_unk(w) for w in sent ] 
    for sent in sentences
]

X = [ np.asarray(sent[:-1]) for sent in sentences_ix ]
Y = [ np.asarray(sent[1:]) for sent in sentences_ix ] 

print('Ejemplos X')
print(X[:3])
print('Ejemplos Y')
print(Y[:3])

Ejemplos X
[array([5272,  239,    2,   11,  731,    0,   53, 1472,   19, 5272,    8,
       1473,    7,  875]), array([   8,  208,    8, 5272,    0,  330,    2,    3,   26,    1,    3,
        110,    0,   46,    7, 1474,    9, 1475,   48,  218,    7,  170,
          2,   54,   20,  160,    0,   54,  219,    1,    4,  161,    1,
          3,  111]), array([ 124,   16,    4, 1476,    2,   31,    0,    2, 5272,   34,  331,
          2,   44,    5,  732,    5,  473,   15,    0,   35, 1477])]
Ejemplos Y
[array([ 239,    2,   11,  731,    0,   53, 1472,   19, 5272,    8, 1473,
          7,  875,    6]), array([ 208,    8, 5272,    0,  330,    2,    3,   26,    1,    3,  110,
          0,   46,    7, 1474,    9, 1475,   48,  218,    7,  170,    2,
         54,   20,  160,    0,   54,  219,    1,    4,  161,    1,    3,
        111,    6]), array([  16,    4, 1476,    2,   31,    0,    2, 5272,   34,  331,    2,
         44,    5,  732,    5,  473,   15,    0,   35, 1477,    6])]


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30)

Mando los vectores de entrada y salida a tensores en gpu

In [22]:
def to_pytorch_tensor(list_of_lists):
    return [
        torch.from_numpy(l).long().to(device)
        for l in list_of_lists
    ]

In [23]:
X_train = to_pytorch_tensor(X_train)
X_test = to_pytorch_tensor(X_test)
Y_train = to_pytorch_tensor(Y_train)
Y_test = to_pytorch_tensor(Y_test)

## Modelo

### 1. Capa de embedding

### 2. Capa oculta

### 3. Capa de salida

Defino las variables para la red neuronal

In [85]:
# Dimensión de entrada (one-hot), tamaño del vocabulario
D_in = len(index_to_w)

# Dimensión de la capa de embedding
D_emb = 16 # 32

# Dimensión de la capa lstm
D_lstm = 8 # 16

# Dimensión de la capa de salida
D_out = D_in

# Épocas de entrenamiento
num_epochs = 10

# Learning rate
lr = 0.001

# Betas para Adam
beta1 = 0.0002
beta2 = 0.999

In [106]:
class Model(nn.Module):
    def __init__(self, ngpu, D_in, D_emb, D_lstm, D_out):
        super(Model, self).__init__()
        self.ngpu = ngpu
        self.embedding = nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb)#, padding_idx=0)
        self.lstm = nn.LSTM(input_size=D_emb, hidden_size=D_lstm, bias=True)#, batch_first=True)
        # self.out_layer = nn.Sequential(
        #     nn.Embedding(num_embeddings=D_in, embedding_dim=D_emb, padding_idx=0),
        #     nn.LSTM(input_size=D_emb, hidden_size=D_lstm, bias=True, batch_first=True),
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=D_out)
        # )
        self.linear = nn.Linear(in_features=D_lstm, out_features=D_out, bias=True)
        # self.out_layer = nn.Softmax(dim=D_out)
        # self.out_layer = nn.Sequential(
        #     nn.Linear(in_features=D_lstm, out_features=D_out, bias=True),
        #     nn.Softmax(dim=D_out)
        # )

    def forward(self, sentence):
        T = len(sentence)
        # print(T)
        embeddings = self.embedding(sentence).view(T, 1, -1)
        # print('Embeddings')
        # print(embeddings)
        lstm_out, (ht, ct) = self.lstm(embeddings)
        lstm_out = lstm_out.view(T, -1)
        # print('LSTM_out')
        # print(lstm_out)
        preact_out = self.linear(lstm_out).view(T, -1)
        # print('Preact out')
        # print(preact_out)
        return F.log_softmax(preact_out, dim=1)

In [107]:
model = Model(ngpu, D_in, D_emb, D_lstm, D_out).to(device)
model

Model(
  (embedding): Embedding(2292, 16)
  (lstm): LSTM(16, 8)
  (linear): Linear(in_features=8, out_features=2292, bias=True)
)

In [108]:
# # Inicialización de pesos
# def weights_init(m):
#     classname = m.__class__.__name__
#     if classname.find('Embedding') != -1:
#         # Regularizo los pesos
#         n = m.num_embeddings
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#     elif classname.find('Linear') != -1:
#         n = m.in_features
#         y = 1.0/np.sqrt(n)
#         m.weight.data.uniform_(-y, y)
#         m.bias.data.fill_(0)
#     elif classname.find('LSTM') != -1:
#         n = m.input_size
#         y = 1.0/np.sqrt(n)
#         m.weight_ih_l.data.fill_(0)
#         m.weight_hh_l.data.uniform_(-y, y)
#         m.bias_ih_l.data.fill_(0)
        # m.bias_hh_l.data.fill_(0)

In [109]:
# model.apply(weights_init)

Entropía cruzada como optmizador y SGD como optimizador

In [110]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr, betas=(beta1, beta2))
# optimizer = optim.SGD(model.parameters(), lr=0.1)

### Entrenamiento

In [111]:
for epoch in range(num_epochs):
    for x, y in zip(X_train, Y_train):
        # Limpiamos gradientes acumulados
        model.zero_grad()

        # Forward
        # print(x)
        pred = model(x)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch:>12} Loss: {loss}')


Embeddings
tensor([[[-1.7881e-01, -4.5229e-01, -9.4707e-01, -3.7844e-01, -3.0049e-01,
           1.7676e+00,  1.8660e-01,  8.9402e-01, -2.9348e-01, -3.2718e-01,
           5.9819e-01, -1.3414e-01,  7.3961e-01, -8.5819e-01, -1.0146e-02,
           2.0443e-01]],

        [[-6.7835e-01, -9.1612e-01,  8.7945e-01,  3.9427e-02, -9.7080e-02,
           8.5375e-01,  1.0209e+00, -2.0906e+00, -1.1392e+00, -3.3436e-02,
          -9.5894e-01,  9.1496e-01,  3.0243e-01,  3.8461e-01, -2.3871e-01,
           8.2738e-02]],

        [[-1.6273e+00,  1.4199e+00,  3.5919e-01, -4.7290e-01, -5.1169e-01,
          -7.7602e-01,  2.2123e+00,  7.3892e-01,  7.6709e-01, -2.4265e-01,
           9.2501e-02,  6.1254e-02, -6.7082e-02, -6.4032e-01, -3.4414e-01,
          -6.2533e-01]],

        [[ 1.1072e+00, -6.2959e-01,  1.5976e+00, -1.0305e+00, -2.0247e+00,
           1.4729e+00,  6.8229e-01,  9.6242e-01,  1.3576e+00, -1.5342e+00,
           2.5808e-01,  6.1571e-01,  6.6180e-01, -1.2623e+00, -1.2223e+00,
          -